Cloning the data from github repository

In [2]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 38.53 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


Storing the data in the form of dictionary where key represents 8 different emotions which are strings and value stores the image names in the form of lists

In [3]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f :
  reader= csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]



In [7]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

Making directory and sub-directory

In [8]:
import os
os.mkdir('master_data1')
os.mkdir('master_data1/training')
os.mkdir('master_data1/testing')

In [9]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data1/training/',emotion))
  os.mkdir(os.path.join('master_data1/testing/',emotion))

Copying data in sub directories

In [10]:
from shutil import copyfile
split_size=0.8

for emotion, images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data1/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    source = os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data1/testing',emotion,image)
    copyfile(source,dest)


Started building CNN model

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten


In [19]:
from keras.api._v2.keras import activations
model=tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024,activation='relu'),
    Dense(8,activation='Softmax')
])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 10, 10, 64)      

In [15]:
train_dir='/content/master_data1/training'
test_dir='/content/master_data1/testing'
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(
                                                  train_dir,
                                                  target_size=(100,100),
                                                  class_mode='categorical',
                                                  batch_size=128
                                                 )

test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=train_datagen.flow_from_directory(
                                                  test_dir,
                                                  target_size=(100,100),
                                                  class_mode='categorical',
                                                  batch_size=128
                                                 )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [17]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

Training the model


In [20]:
model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es])

<ipython-input-20-860791d9b115>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 21s 223ms/step - loss: 0.9865 - acc: 0.5835 - val_loss: 1.3767 - val_acc: 0.6233
Epoch 2/10
86/86 [==============================] - 20s 231ms/step - loss: 0.6047 - acc: 0.7956 - val_loss: 1.9446 - val_acc: 0.6667
Epoch 3/10
86/86 [==============================] - 19s 220ms/step - loss: 0.5189 - acc: 0.8186 - val_loss: 2.0284 - val_acc: 0.6714
Epoch 4/10
86/86 [==============================] - 22s 253ms/step - loss: 0.4531 - acc: 0.8400 - val_loss: 2.2913 - val_acc: 0.6783
Epoch 5/10
86/86 [==============================] - 20s 229ms/step - loss: 0.4135 - acc: 0.8559 - val_loss: 2.0895 - val_acc: 0.6772
Epoch 6/10
86/86 [==============================] - 19s 224ms/step - loss: 0.3833 - acc: 0.8653 - val_loss: 2.4970 - val_acc: 0.6703
